In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
from scipy import constants
from numpy.polynomial import Polynomial
import pandas as pd

In [ ]:
array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
cavity_E_array_0_00 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0.npy")
cavity_E_array_0_001 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_001.npy")
cavity_E_array_0_005 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_005.npy")
cavity_E_array_0_01 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_01.npy")
cavity_E_array_0_02 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_02.npy")
cavity_E_array_0_03 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_03.npy")
cavity_E_array_0_04 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_04.npy")
cavity_E_array_0_05 = np.load(array_data + "/HHe/fci_cavity_arrays_HHe_6311G0_05.npy")
r_data = np.load(array_data + "/HHe/fci_r_array_HHe.npy")
N_R = r_data.shape[0]
print(N_R)

## Calculation of k
Fit ground state PES of H2 to a quintic polynomial

In [ ]:
def find_crossover(array1, array2):
    idx = (np.abs(array1 - array2)).argmin()

    return idx

def compute_full_array(array1, array2, idx1, idx2):
    index = find_crossover(array1[idx1,:], array2[idx2,:])
    array_1 = array1[idx1,:index]
    array_2 = array2[idx2,index:]
    array_full = np.concatenate((array_1, array_2), axis=0)    

    return array_full

S0_array = cavity_E_array_0_00[0,:]
S1_array = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 3, 2)

LP_0_00 = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 1, 2)
LP_0_001 = compute_full_array(cavity_E_array_0_001, cavity_E_array_0_001, 1, 2)
LP_0_005 = compute_full_array(cavity_E_array_0_005, cavity_E_array_0_005, 1, 2)
LP_0_01 = compute_full_array(cavity_E_array_0_01, cavity_E_array_0_01, 1, 2)
LP_0_02 = compute_full_array(cavity_E_array_0_02, cavity_E_array_0_02, 1, 2)
LP_0_03 = compute_full_array(cavity_E_array_0_03, cavity_E_array_0_03, 1, 2)
LP_0_04 = compute_full_array(cavity_E_array_0_04, cavity_E_array_0_04, 1, 2)
LP_0_05 = compute_full_array(cavity_E_array_0_05, cavity_E_array_0_05, 1, 2)

UP_0_00 = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 3, 3)
UP_0_001 = compute_full_array(cavity_E_array_0_001, cavity_E_array_0_00, 3, 3)
UP_0_005 = compute_full_array(cavity_E_array_0_005, cavity_E_array_0_005, 3, 3)
UP_0_01 = compute_full_array(cavity_E_array_0_01, cavity_E_array_0_01, 3, 3)
UP_0_02 = compute_full_array(cavity_E_array_0_02, cavity_E_array_0_02, 3, 3)
UP_0_03 = compute_full_array(cavity_E_array_0_03, cavity_E_array_0_03, 3, 3)
UP_0_04 = compute_full_array(cavity_E_array_0_04, cavity_E_array_0_04, 3, 3)
UP_0_05 = compute_full_array(cavity_E_array_0_05, cavity_E_array_0_05, 3, 3)

LP_arrays = np.array([LP_0_00, 
            LP_0_001,
            LP_0_005,
            LP_0_01,
            LP_0_02,
            LP_0_03,
            LP_0_04,
            LP_0_05]
             )

UP_arrays = np.array([UP_0_00, 
            UP_0_001,
            UP_0_005,
            UP_0_01,
            UP_0_02,
            UP_0_03,
            UP_0_04,
            UP_0_05]
             )

In [ ]:

min_LP_array = np.zeros((1,8))
min_UP_array = np.zeros((1,8))

for i in range(8):
    min_LP_array[0,i] = np.argmin(LP_arrays[i])
    min_UP_array[0,i] = np.argmin(UP_arrays[i])

min_S0_loc = np.argmin(S0_array)
min_S1_loc = np.argmin(S1_array)

r_eq_ang = r_data[min_S0_loc]
print(f'Min on S0 is {r_data[min_S0_loc]}')
print(f'Min on S1 is {r_data[min_S1_loc]}')
#print(f'Min on LP is {r_data[min_LP_loc]}')
#print(f'Min on UP is {r_data[min_UP_loc]}')

# Fitting S0 PES to a quintic polynomial
au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
poly = np.poly1d(np.polyfit(r_data, S0_array, 10))
print(poly)

#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = poly.deriv()
second_derivative = first_derivative.deriv()
k_test_au = second_derivative(r_eq_ang)
k_test_SI = k_test_au * au_to_SI
print(k_test_SI)

#plotting S0 PES and quintic fit
plt.plot(r_data, poly(r_data), 'm-', label='fit')
plt.plot(r_data, S0_array, 'bo', label='cavity free |g>')
plt.show

## Calculation of $ \text{x}_0 $

$$ \frac{\hbar}{2}\sqrt{\frac{k}{\mu}} = \frac{k}{2}(x_0 - x_{eq})^2 + V_0 $$

Expanded, solved for $ x_o $, and found zeros using quadratic formula

In [ ]:
mu_amu = 1.00784 * 1.00784 / (1.00784 + 1.00784)
mu_kg = mu_amu  * 10 ** (-3) / (6.022 * 10 ** 23) 
r_eq_SI = r_eq_ang * 10 ** (-10)
h_bar = constants.hbar
V_0_loc = np.argmin(S0_array)
V_0 = S0_array[V_0_loc] * 4.35974 * 10 ** (-18)
left = (h_bar / 2) * np.sqrt(k_test_SI / mu_kg)
a = 0.5 * k_test_SI 
b = -k_test_SI * r_eq_SI
c = 0.5 * k_test_SI * (r_eq_SI ** 2) - left
zeros_n = (-b - np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
zeros_p = (-b + np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
x0_angstrom = zeros_n * 10 ** 10
x0_au = x0_angstrom / psi4.constants.bohr2angstroms
print(x0_angstrom)

## Calculation of the Huang-Rhys Factor
Huang Rhys factor can be calculated by both

$$ S = 1/2(\Delta x / x_0)^2 \tag{Turner}$$

from the mode anharmonicity paper

and 

$$ S = \frac{m\omega_{vib} \Delta x^2}{2 \hbar} \tag{Hsu}$$

from the polaritonic Huang-Rhys factor paper

In [ ]:
# Constants and Variables
delta_au = (r_data[min_S1_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms

delta_LP_au = np.zeros((1,8))
delta_UP_au = np.zeros((1,8))

for i in range(8):
    delta_LP_au[0,i] = (r_data[int(min_LP_array[0,i])] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
    delta_UP_au[0,i] = (r_data[int(min_UP_array[0,i])] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms


delta_m = (r_data[min_S1_loc] - r_data[min_S0_loc]) * 10 ** (-10)
delta_angstrom = (r_data[min_S1_loc] - r_data[min_S0_loc])
omega_vib = np.sqrt(k_test_SI / mu_kg)
h_bar = constants.hbar
x0_test = np.sqrt(h_bar * omega_vib / k_test_SI)
x0_test_au = (x0_test * 10 ** (10))  / psi4.constants.bohr2angstroms

# Turner
S_Turner = 0.5 * (delta_au / x0_test_au) ** 2

# Hsu
S_Hsu = mu_kg * omega_vib * delta_m ** 2 / (2 * h_bar)

#g and LP 

HR_LP_array = np.zeros((1,8))
HR_UP_array = np.zeros((1,8))

for i in range(8):
    HR_LP_array[0,i] = 0.5 * (delta_LP_au[0,i] / x0_test_au) ** 2
    HR_UP_array[0,i] = 0.5 * (delta_UP_au[0,i] / x0_test_au) ** 2

print('|g> and |e> HR Factor')
print(S_Turner)
print(S_Hsu)

print('LP and UP HR Factor')
print(HR_LP_array)
print(HR_UP_array)


df = pd.DataFrame(HR_LP_array.reshape(-1,1))
df.to_clipboard()

First plot the ground-state potential energy surfaces for $ \text{H2} $ inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:
fci_S0_omega = np.zeros((1, N_R))
for i in range(N_R):
    fci_S0_omega[0,i] = S0_array[i] + 0.7154401621865771
fci_S0_plusw = np.ndarray.flatten(fci_S0_omega)

plt.plot(r_data, fci_S0_plusw, 'r-', label='cavity free |g + $\omega$>')
plt.plot(r_data, S1_array, 'b-', label='cavity free |e>')
plt.plot(r_data, cavity_E_array_0_05[3,:])
for i in range(8):
    plt.plot(r_data, LP_arrays[i,:])
    plt.plot(r_data, UP_arrays[i,:])
plt.xlabel('r (Angstrom)')
plt.ylabel('E (Hartree)')


#plt.xlim(0.8, 1.2)
#plt.ylim(-2.25, -2.15)
plt.legend()
plt.savefig('ppsurfaces.png',dpi=500)
#plt.show()

In [ ]:
#pd.DataFrame(matrix).to_clipboard()

omega_test = np.abs(S1_array[40] - cavity_E_array_0_00[0,40])
print(omega_test)
print(r_data[40])
